In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp

# 读入文件
df_train = pd.read_csv('ua.base', sep='\t')
df_test = pd.read_csv('ua.test', sep='\t')

n_users = df_train.uid.unique().shape[0]
n_items = df_train.mid.unique().shape[0]
print('Number of users = %d | Number of movies = %d' % (n_users, n_items))

Number of users = 943 | Number of movies = 1680


In [21]:
# 使用三元组(ratings, (uid, mid))初始化一个稀疏矩阵
base_data_matrix = sp.coo_matrix((df_train.rating.values, (df_train.uid.values, df_train.mid.values))).toarray()
test_data_matrix = sp.coo_matrix((df_test.rating.values, (df_test.uid.values, df_test.mid.values))).toarray()

pred_mat = np.zeros_like(test_data_matrix)

# 计算每一位用户的评分均值vector
average = []
for k in range(n_users+1):
    a= base_data_matrix[k]
    average.append(np.sum(a)/len((np.nonzero(a)[0])))

c:\users\jingc\anaconda3\envs\pythonproject1\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  # Remove the CWD from sys.path while we load stuff.


In [22]:
average = np.array(average)

In [24]:

#使用波动+均值来提高了baseline模型
eps = np.finfo(float).eps
for i in range(n_users):
    ln = base_data_matrix[i]
    tn = test_data_matrix[i]
    # Jaccard相似度，分母加上eps，避免除以0
    jaccard = np.sum(np.logical_and(ln, base_data_matrix), axis=1) / (
                np.sum(np.logical_or(ln, base_data_matrix), axis=1) + eps)
    nz = np.nonzero(tn)[0]  # 找出要所有预测的items的id
    for j in nz:
        ref_ratings = base_data_matrix[:, j]  # 找出所有对item j所有的ratings
        ref_users = np.nonzero(ref_ratings)  # 找出对应的user
        jc_ref = jaccard[ref_users]  # 找出对应的similarities
        avgrating_u = np.sum(ln)/len((np.nonzero(ln)[0]))#计算被预测用户的评分均值
        diff_v = ref_ratings[ref_users]-average[ref_users]#对应user的评分减去 对应user的评分均值
        pred_rating = avgrating_u + np.sum(jc_ref *diff_v)/(np.sum(jc_ref) + eps)
        pred_mat[i, j] = pred_rating

# 计算MAE
N = np.count_nonzero(test_data_matrix)
MAE = np.sum(np.abs(pred_mat - test_data_matrix)) / N
print('User-based CF MAE: %g' % MAE)

User-based CF MAE: 0.860233


## 相比原来的0.94提升了8.5%